<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC157.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [AtCoder Beginner Contest 157](https://atcoder.jp/contests/abc157)

## [A - Douplex Printing](https://atcoder.jp/contests/abc157/tasks/abc157_a)

In [ ]:
N = int(input())
print(-(-N//2))

## [B - Bingo](https://atcoder.jp/contests/abc157/tasks/abc157_b)

In [ ]:
A = [list(map(int, input().split())) for _ in range(3)]
N = int(input())
num = set([int(input()) for _ in range(N)])

In [ ]:
flag = False
# 縦 & 横
for i in range(3):
    if all(A[i][j] in num for j in range(3)):
        flag = True
    if all(A[j][i] in num for j in range(3)):
        flag = True

In [ ]:
# 斜め
if all(A[i][i] in num for i in range(3)):
    flag = True
if (A[1][1] in num) and (A[0][2] in num) and (A[2][0] in num):
    flag = True

In [ ]:
print("Yes" if flag else "No")

Yes


## [C - Guess The Number](https://atcoder.jp/contests/abc157/tasks/abc157_c)

In [8]:
N, M = map(int, input().split())
cond = [list(map(int, input().split())) for _ in range(M)]

3 1
1 0


In [9]:
if N == 1:
    start = 0
else:
    start = 10**(N-1)

for num in range(start, 10**N):
    num = str(num)
    flag = True
    for s, c in cond:
        if len(num) < s:
            flag = False
            break
        if num[s - 1] != str(c):
            flag = False
            break    
    if flag:
        print(num)
        break
else:
    print(-1)

0


## [D - Friend Suggestions](https://atcoder.jp/contests/abc157/tasks/abc157_d)

In [ ]:
class UnionFindTree:
    def __init__(self, n):
        self.parents = [-1] * n
  
    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.find(self.parents[x])
  
    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        if x == y: return    
        if self.parents[x] > self.parents[y]:
            x, y = y, x
        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def same_check(self, x, y):
        return self.find(x) == self.find(y)

    def size(self, x):
        return -1 * self.parents[self.find(x)]

In [ ]:
N, M, K = map(int, input().split())
uf = UnionFindTree(N)

friend_graph = [[] for _ in range(N)]
block_graph = [[] for _ in range(N)]
for _ in range(M):
    a, b = map(int, input().split())
    a -= 1
    b -= 1
    uf.union(a, b)
    friend_graph[a].append(b)
    friend_graph[b].append(a)

for _ in range(K):
    c, d = map(int, input().split())
    c -= 1
    d -= 1
    block_graph[c].append(d)
    block_graph[d].append(c)

10 9 3
10 1
6 7
8 2
2 5
8 4
7 3
10 9
6 4
5 8
2 6
7 5
3 1


In [ ]:
for i in range(N):
    ans = uf.size(i) - 1 - len(friend_graph[i])
    for b in block_graph[i]:
        if uf.same_check(i, b):
            ans -= 1
    print(ans)

1
3
5
4
3
3
3
3
1
0


## [E - Simple String Queries](https://atcoder.jp/contests/abc157/tasks/abc157_e)

### Seg木を使う

N ~ 5*10^5


Q ~ 2 * 10^4

In [ ]:
from collections import defaultdict
N = int(input())
S = list(input())
Q = int(input())

7
abcdbbd
6


In [ ]:
A = [1 << (ord(s) - 97) for s in S]

In [ ]:
class SegmentTree:
    """ 1点更新・区間集約 Segment Tree. 非再起・1-index・非可換

    Attributes:
        tree (List[Union[int, float]]): セグメントツリーを表す配列
        segfunc (Callable): 集約を行う関数 (モノイド上の演算)
        ide_ele (float): 集約の関数における単位元　（モノイド上の単位元）
        update (int, float): 元の配列の更新 & セグメントツリーの更新
        query (int, int): 区間集約値の取得
    """
    def __init__(self, n: int, ide_ele: float, seg_func: callable):
        """
        Args:
            n (int): 元の配列のサイズ
            ide_ele (float): モノイドにおける単位元
            segfunc (callable): 集約を行う関数 （モノイド上の演算）
        """
        self.segfunc = seg_func
        self.ide_ele = ide_ele
        self.num = 1 << (n - 1).bit_length()
        self.tree = [self.ide_ele] * (2 * self.num)
    
    def update(self, i: int, x: float) -> None:
        """元の配列のi番目を x に変更（上書き）する. それに伴ってセグメントツリーも更新する. +=のようなupdateではない.

        Args:
            i (int): 元の配列のi番目の要素. 元の配列の0-indexの
            x (float): 変更後の値        
        """
        i += self.num
        self.tree[i] = x
        while i > 1:
            i >>= 1
            self.tree[i] = self.segfunc(self.tree[i << 1], self.tree[(i << 1) + 1])
        
    def query(self, l: int, r: int):
        """元の配列の閉区間[l, r]のsegfuncにおける集約値を取得する

        Args:
            l (int): 元の配列の集約を行う区間の左端のindex. 0-index
            r (int): 元の配列の集約を行う区間の右端のindex. 0-index
        """
        # [l, r) として葉から計算していく
        l += self.num
        r += self.num + 1

        l_res = self.ide_ele
        r_res = self.ide_ele
        
        while l < r:
            if l & 1:  # lが奇数だったらそのノードを集約する
                l_res = self.segfunc(l_res, self.tree[l])
                l += 1  # 右に進む
            if r & 1:  # rが奇数だったら, rは開区間なので その1つ左ノードを集約する
                r_res = self.segfunc(self.tree[r - 1], r_res)

            l >>= 1  # 上に進む（２で割って切り捨て）
            r >>= 1
        
        return self.segfunc(l_res, r_res)

In [ ]:
def popcount(x):
    '''xの立っているビット数をカウントする関数
    (xは64bit整数)'''

    # 2bitごとの組に分け、立っているビット数を2bitで表現する
    x = x - ((x >> 1) & 0x5555555555555555)

    # 4bit整数に 上位2bit + 下位2bit を計算した値を入れる
    x = (x & 0x3333333333333333) + ((x >> 2) & 0x3333333333333333)

    x = (x + (x >> 4)) & 0x0f0f0f0f0f0f0f0f # 8bitごと
    x = x + (x >> 8) # 16bitごと
    x = x + (x >> 16) # 32bitごと
    x = x + (x >> 32) # 64bitごと = 全部の合計
    return x & 0x0000007f

In [ ]:
seg = SegmentTree(N, 0, lambda x, y: x | y)

In [ ]:
for i in range(N):
    seg.update(i, A[i])

In [ ]:
for _ in range(Q):
    query = list(input().split())
    if query[0] == "1":
        i, c = int(query[1]), str(query[2])
        seg.update(i-1, 1 << (ord(c) - 97))
    else:
        l, r = int(query[1]), int(query[2])
        ans = seg.query(l-1, r-1)
        print(popcount(ans))

### Binary Searchで頑張る
計算量的には$O(N^2)$な気がする。。。。

In [ ]:
from bisect import bisect_left, bisect_right, insort
from collections import defaultdict

N = int(input())
S = list(input())
Q = int(input())

cnt = defaultdict(list)
for i, s in enumerate(S):
    cnt[s].append(i)

7
abcdbbd
6


In [ ]:
for _ in range(Q):
    query = list(input().split())
    if query[0] == "1":
        i, c = int(query[1]), query[2]
        i -= 1
        s = S[i]

        if c == S[i]: continue
        
        cnt[s].remove(i)
        S[i] = c
        insort(cnt[c], i)
    
    else:
        l, r = int(query[1]), int(query[2])
        ans = 0
        for i in range(26):
            s = chr(ord("a") + i)
            
            left_ind = bisect_left(cnt[s], l-1)
            right_ind = bisect_right(cnt[s], r-1)
            if right_ind > left_ind:
                ans += 1
        print(ans)

2 3 6
3
1 5 z
2 1 1
1
1 4 a
1 7 d
2 1 7
5
